**Init Communication**


In [1]:
import compas_rrc as rrc
import json

from compas.geometry import Point, Vector, Frame, Translation, Rotation

ros = rrc.RosClient()
ros.run()

filma = rrc.AbbClient(ros, "/rob2")
morda = rrc.AbbClient(ros, "/rob1")

connect_filma = filma.send_and_wait(rrc.PrintText("Connecting to Filma..."))
connect_morda = morda.send_and_wait(rrc.PrintText("Connecting to Morda..."))

if connect_filma == "Done" and connect_morda == "Done":
    print("Connected to both robots successfully.")
else:
    print("Failed to connect to one or both robots.")
    exit(1)

def go_home_mortadela():
    return morda.send_and_wait(rrc.MoveToJoints(joints=[-90,-60,60,0,0,0],ext_axes=None,speed=2000,zone=rrc.Zone.FINE))

def go_home_filma():
    return filma.send_and_wait(rrc.MoveToJoints(joints=[90,-60,60,0,0,0],ext_axes=None,speed=2000,zone=rrc.Zone.FINE))

def idle_mordadela():
    return morda.send_and_wait(rrc.MoveToJoints(joints=[-75, -30, 60, 90, 90, 0],ext_axes=None,speed=2000,zone=rrc.Zone.FINE))

def idle_filma():
    return filma.send_and_wait(rrc.MoveToJoints(joints=[75, -30, 60, 90, -60, 0],ext_axes=None,speed=2000,zone=rrc.Zone.FINE))

def open_gripper_mordadela():
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 1)) # OPEN
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO6", 0)) # OPEN

def close_gripper_mordadela():
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 0)) # CLOSE
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO6", 1)) # CLOSE

def open_gripper_filemona():
    filma.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 1)) # OPEN
    filma.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO4", 0)) # OPEN    

def close_gripper_filemona():
    filma.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 0)) # CLOSE
    filma.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO4", 1)) # CLOSE


Connected to both robots successfully.


In [ ]:
# # ─── Cell: Read Active TCP for Filma & Morda ────────────────────────────

# # Filma: set tool and get TCP frame

# done = filma.send_and_wait(rrc.SetTool('fingergrabfi'))
# filma_tcp_frame = filma.send_and_wait(rrc.GetFrame())
# print("Active TCP frame:", filma_tcp_frame)

# # Morda: set tool and get TCP frame

# done = morda.send_and_wait(rrc.SetTool('fingergrab'))
# morda_tcp_frame = morda.send_and_wait(rrc.GetFrame())
# print("Active TCP frame:", morda_tcp_frame)



In [2]:
# ─── Cell 2 (updated): pick_and_place WITHOUT home_fn ───────────────────
from compas.geometry import Translation

def pick_and_place(
    robot,
    pick_frame,
    place_frame,
    # waypoint_frame,
    speed,
    zone,
    open_fn,
    close_fn,
    idle_fn
):
    # Build 150 mm–back “approach” poses
    approach_pick  = pick_frame.transformed(Translation.from_vector(pick_frame.zaxis * -300))
    approach_place = place_frame.transformed(Translation.from_vector(place_frame.zaxis * -300))

    # 1) Approach → open → pick → close → retract
    robot.send_and_wait(rrc.MoveToFrame(approach_pick, speed, zone, rrc.Motion.JOINT))
    # open_fn()
    robot.send_and_wait(rrc.MoveToFrame(pick_frame,   speed/5.0, zone, rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    # close_fn()
    robot.send_and_wait(rrc.MoveToFrame(approach_pick, speed, zone, rrc.Motion.LINEAR))

    # 2) Via waypoint
    # robot.send_and_wait(rrc.MoveToFrame(waypoint_frame, speed, zone, rrc.Motion.JOINT))

    # 3) Approach place → open → place → close → retract
    robot.send_and_wait(rrc.MoveToFrame(approach_place, speed, zone, rrc.Motion.JOINT))
    # open_fn()
    robot.send_and_wait(rrc.MoveToFrame(place_frame,    speed/5.0, zone, rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    # close_fn()
    robot.send_and_wait(rrc.MoveToFrame(approach_place, speed, zone, rrc.Motion.LINEAR))

    # 4) Back to idle
    idle_fn()


In [3]:
# --- Manual frame definitions ---
# Filma (rob2) pick & place (same for all blocks)
filma_pick_frame  = Frame(Point(232.82, 2275.88, 801.48), Vector(-0.9948, -0.0974, -0.0288), Vector(-0.0988, 0.9936, 0.0544))
filma_place_frame = Frame(Point(1535.78, 1502.34, 1020.33), Vector(-0.99506, -0.09126, -0.03909), Vector(-0.09190, 0.99566, 0.01482))

# Morda (rob1) pick frame (fixed)
morda_pick_frame  = Frame(Point(1717.60, -1490.24, 988.89), Vector(-0.99890, -0.02895, 0.03699), Vector(-0.02878, 0.99957, 0.00506))

# Morda place frames (one per block)
morda_place_frames = [
    Frame(Point(1582.75, 123.95, 978.88), Vector(-0.10008, -0.99498, -0.00266), Vector(-0.99433, 0.09992, 0.03639)),
    # ... repeat and fill in up to 20 entries ...
]

# Number of blocks
num_blocks = len(morda_place_frames)

# Derive lists for Filma (repeat same frame for each block)
pick_frames_filma  = [filma_pick_frame] * num_blocks
place_frames_filma = [filma_place_frame] * num_blocks

In [4]:
# Home_check: Send both robots to home before starting any cycles ────────
print("Homing both robots…")
go_home_filma()
go_home_mortadela()
print("Both robots are at home.")


Homing both robots…
Both robots are at home.


In [5]:
# ─── Cell Z: Two-robot cycles without per-cycle homing, then final home ──
num_parts = len(place_frames_filma)
assert num_parts == len(pick_frames_filma) == len(morda_pick_frame) == len(morda_place_frames)

for i in range(num_parts):
    block = i + 1
    print(f"\n=== Cycle {block} of {num_parts} ===")

    print(f"Filma starting block {block}")
    pick_and_place(
        robot          = filma,
        pick_frame     = pick_frames_filma[i],
        place_frame    = place_frames_filma[i],
        # waypoint_frame = waypoint_frame_filma,
        speed          = 2000,
        zone           = rrc.Zone.FINE,
        # open_fn        = open_gripper_filemona,
        # close_fn       = close_gripper_filemona,
        idle_fn        = idle_filma
    )
    print(f"Filma finished block {block}")

    print(f"Morda starting block {block}")
    pick_and_place(
        robot          = morda,
        pick_frame     = morda_pick_frame[i],
        place_frame    = morda_place_frames[i],
        # waypoint_frame = waypoint_frame_morda,
        speed          = 2000,
        zone           = rrc.Zone.FINE,
        # open_fn        = open_gripper_mordadela,
        # close_fn       = close_gripper_mordadela,
        idle_fn        = idle_mordadela
    )
    print(f"Morda finished block {block}")

print("\nAll cycles done — sending both robots home again.")
go_home_filma()
go_home_mortadela()
print("Both robots are homed. Sequence complete.")


AssertionError: 